In [1]:
import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt, eval_single_ckpt
from noise import add_noise_to_weights
import numba
import logging
import time

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import torch
import torch.distributed as dist
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models_multinomial import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
from train_utils.train_utils import train_model, model_save
from eval_utils import eval_utils_multinomial
import numpy as np

import matplotlib.pyplot as plt
import matplotlib


matplotlib.use('Agg')

from hardware_noise.new_weight_mapping import weights_mapping as add_noise_to_weights


# from datetime import datetime
print(datetime.datetime.now())

def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='cfgs/kitti_models/pointpillar.yaml', \
                        help='specify the config for training')

    parser.add_argument('--batch_size', type=int, default=None, required=False, help='batch size for training')
    parser.add_argument('--epochs', type=int, default=None, required=False, help='number of epochs to train for')
    parser.add_argument('--workers', type=int, default=32, help='number of workers for dataloader')
    parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    parser.add_argument('--ckpt', type=str, default='checkpoint_epoch_80.pth', \
                        help='checkpoint to start from')
    parser.add_argument('--pretrained_model', type=str, default=None, help='pretrained_model')
    parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    parser.add_argument('--tcp_port', type=int, default=29051, help='tcp port for distrbuted training')
    parser.add_argument('--sync_bn', action='store_true', default=False, help='whether to use sync bn')
    parser.add_argument('--fix_random_seed', action='store_true', default=True, help='')
    parser.add_argument('--ckpt_save_interval', type=int, default=1, help='number of training epochs')
    parser.add_argument('--local_rank', type=int, default=0, help='local rank for distributed training')
    parser.add_argument('--max_ckpt_save_num', type=int, default=30, help='max number of saved checkpoint')
    parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER,
                        help='set extra config keys if needed')

    parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    parser.add_argument('--start_epoch', type=int, default=0, help='')
    parser.add_argument('--save_to_file', action='store_true', default=False, help='')
    
    # parser.add_argument('--local_rank', type=int, default=-1, metavar='N', help='Local process rank.')

    args = parser.parse_known_args()[0]

    cfg_from_yaml_file(args.cfg_file, cfg)
    cfg.TAG = Path(args.cfg_file).stem
    cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'

    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs, cfg)

    return args, cfg


args, cfg = parse_config()


class Opt():
    def __init__(self, f_name, train_set, train_loader, train_sampler, test_set, test_loader, sampler):
        self.f_name = f_name
        # self.model = model
        self.train_set = train_set
        self.train_loader = train_loader
        self.train_sampler = train_sampler
        self.test_set = test_set
        self.test_loader = test_loader
        self.sampler = sampler
        
    def opt_function(self, p1, p2, p3):
        # f_name = self.f_name
        # model = self.model
        train_set = self.train_set
        train_loader = self.train_loader
        train_sampler = self.train_sampler
        test_set = self.test_set
        test_loader = self.test_loader
        sampler = self.sampler
        
        model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), 
                            p1=p1, 
                            p2=p2, 
                            p3=p3,
                            dataset=train_set)
        model.cuda()
        
        optimizer = build_optimizer(model, cfg.OPTIMIZATION)
        model.load_params_with_optimizer(args.ckpt, to_cpu=dist, optimizer=optimizer, logger=logger)
        
        
        global n
        n += 1
        
        print("=============")
        print(p1, p2, p3)
        print("=============")

        global best_accu
        
        # # load checkpoint if it is possible
        start_epoch = it = 0
        last_epoch = -1

        model.train()  # before wrap to DistributedDataParallel to support fixed some parameters
        model.cuda()

        lr_scheduler, lr_warmup_scheduler = build_scheduler(
            optimizer, total_iters_each_epoch=len(train_loader), total_epochs=args.epochs,
            last_epoch=last_epoch, optim_cfg=cfg.OPTIMIZATION
        )

        # -----------------------start training---------------------------
        logger.info('**********************Start training %s/%s(%s)**********************'
                    % (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))
        
        output_dir = cfg.ROOT_DIR / 'tools' / 'save_path' / args.extra_tag
        ckpt_dir = output_dir / 'ckpt'
        output_dir.mkdir(parents=True, exist_ok=True)
        ckpt_dir.mkdir(parents=True, exist_ok=True)
        print(ckpt_dir)

        # train_model(
        #     model,
        #     optimizer,
        #     train_loader,
        #     model_func=model_fn_decorator(),
        #     lr_scheduler=lr_scheduler,
        #     optim_cfg=cfg.OPTIMIZATION,
        #     start_epoch=start_epoch,
        #     total_epochs=args.epochs,
        #     start_iter=it,
        #     rank=cfg.LOCAL_RANK,
        #     tb_log=tb_log,
        #     ckpt_save_dir=ckpt_dir,
        #     train_sampler=train_sampler,
        #     lr_warmup_scheduler=lr_warmup_scheduler,
        #     ckpt_save_interval=args.ckpt_save_interval,
        #     max_ckpt_save_num=args.max_ckpt_save_num,
        #     merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch
        # )


        ckpt_pth = save_path+'bayes_model-{}-{}-{}'.format(p1, p2, p3)
        ckpt_name = ckpt_pth+'.pth'

        logger.info('**********************End training**********************')

        time.sleep(30)


        f_name = self.f_name
        f = open(save_path+'result.txt', "a+")
        f.write('----------------Noise-{}-evaluate----------------'.format(f_name))
        f.write('----------------{}-{}-{}---------------\n'.format(p1, p2, p3))
        f.close()

        logger.info('---------------Epoch-{}-Noise-{}-evaluate----------------'.format(n, f_name))
        model = add_noise_to_weights('./hardware_noise/hardware_data/'+f_name, 0, f_name, model, 'cuda')

        acc1 = eval_utils_multinomial.eval_simple(args.ckpt, p1, p2, p3, f_name, n, cfg, model, test_loader, logger, save_path, dist_test=dist_train, save_to_file=args.save_to_file, result_dir=eval_output_dir)
        print("----------")
        print(acc1)
        print("----------")
        logger.info('**********************End evaluation**********************')

            # best_accu = acc

        return acc1  #+acc2+acc3


if args.launcher == 'none':
    dist_train = False  # True
    total_gpus = 1
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_gpu = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    print('Using GPU:' + str(np.argmax(memory_gpu)))
    os.environ["CUDA_VISIBLE_DEVICES"] = str(np.argmax(memory_gpu))
    os.system('rm tmp')
else:
    args.is_master = args.local_rank == 0
    args.device = torch.cuda.device(args.local_rank)
    torch.cuda.manual_seed_all(666)
    # dist.init_process_group(backend='nccl', init_method='env://', rank = 0, world_size = 1)
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '5678'
    # dist.init_process_group(backend='nccl', init_method='env://', rank = 0, world_size = 1)
    torch.cuda.set_device(1)
    device = torch.device('cuda', cfg.LOCAL_RANK)
    total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
        args.tcp_port, args.local_rank, backend='nccl'
    )
    dist_train = True


if args.batch_size is None:
    args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
else:
    assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
    args.batch_size = args.batch_size // total_gpus

args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

if args.fix_random_seed:
    common_utils.set_random_seed(666)

output_dir = cfg.ROOT_DIR / 'output' / cfg.EXP_GROUP_PATH / cfg.TAG / args.extra_tag
ckpt_dir = output_dir / 'ckpt'
output_dir.mkdir(parents=True, exist_ok=True)
ckpt_dir.mkdir(parents=True, exist_ok=True)

log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
# logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)
logger = common_utils.create_logger('./baseline/pointpillar/log.txt', rank=cfg.LOCAL_RANK)

file = open('./baseline/pointpillar/result.txt','w')
file.write('results\n')
file.close()


# log to file
logger.info('**********************Start logging**********************')
gpu_list = os.environ['CUDA_VISIBLE_DEVICES'] if 'CUDA_VISIBLE_DEVICES' in os.environ.keys() else 'ALL'
logger.info('CUDA_VISIBLE_DEVICES=%s' % gpu_list)


for key, val in vars(args).items():
    logger.info('{:16} {}'.format(key, val))
log_config_to_file(cfg, logger=logger)
if cfg.LOCAL_RANK == 0:
    os.system('cp %s %s' % (args.cfg_file, output_dir))

tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

# -----------------------create dataloader & network & optimizer---------------------------
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=args.batch_size,
    dist=dist_train, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs
)


logger.info('**********************Start evaluation %s/%s(%s)**********************' %
            (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=args.batch_size,
    dist=dist_train, workers=args.workers, logger=logger, training=False
)
eval_output_dir = output_dir / 'eval' / 'eval_with_train'
eval_output_dir.mkdir(parents=True, exist_ok=True)
args.start_epoch = max(args.epochs - 10, 0)  # Only evaluate the last 10 epochs


logger.info('----------------Noise Experiment----------------')

save_path = './save_path/'
f = open(save_path+'3d.txt', "a+")
f.write(str(datetime.datetime.now())+'\n')
f.close()

f = open(save_path+'3d.txt', "a+")
f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('s', 'n', 'mthd',
                                    'Car/easy_R40',
                                    'Car/moderate_R40',
                                    'Car/hard_R40',
                                    'Pedestrian/easy_R40',
                                    'Pedestrian/moderate_R40',
                                    'Pedestrian/hard_R40',
                                    'Cyclist/easy_R40',
                                    'Cyclist/moderate_R40',
                                    'Cyclist/hard_R40',
                                    'easy_R40',
                                    'moderate_R40',
                                    'hard_R40',
                                    'avg', 'time'
                                    ))
f.close()


hw_data_files = os.listdir('./hardware_noise/hardware_data/')


file2ap_dict = {}
for f_name in sorted(hw_data_files)[-5:]:
    if f_name.endswith('xlsx'):
        file2ap_dict[f_name] = {}
        print(f_name)
        
        print("=============")
        p1 = 0.42
        p2 = 0.23
        p3 = 0.11
        print(p1, p2, p3)
        print("=============")
        
        opt= Opt(f_name, train_set, train_loader, train_sampler, test_set, test_loader, sampler)
        opt_function = opt.opt_function
        
        # Bounded region of parameter space
        pbounds = {'p1': (0.1, 0.9), 'p2': (0.1, 0.9), 'p3': (0.1, 0.9)}


        optimizer_bayes = BayesianOptimization(
            f=opt_function,
            pbounds=pbounds,
            verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
            random_state=1,
        )
        optimizer_bayes.probe(
            params={'p1': 0.42, 'p2': 0.23, 'p3': 0.11},
            lazy=True,
        )

        logger_bayes = JSONLogger(path=save_path+"logs2.json")
        optimizer_bayes.subscribe(Events.OPTIMIZATION_STEP, logger_bayes)


        n = 0
        optimizer_bayes.maximize(
            init_points=3,
            n_iter=10,
        )
logger.info('**********************End evaluation %s/%s(%s)**********************')



#################### importing ##########################


2023-07-05 15:07:19,261   INFO  **********************Start logging**********************
2023-07-05 15:07:19,262   INFO  CUDA_VISIBLE_DEVICES=0
2023-07-05 15:07:19,262   INFO  cfg_file         cfgs/kitti_models/pointpillar.yaml
2023-07-05 15:07:19,263   INFO  batch_size       4
2023-07-05 15:07:19,263   INFO  epochs           80
2023-07-05 15:07:19,264   INFO  workers          32
2023-07-05 15:07:19,264   INFO  extra_tag        default
2023-07-05 15:07:19,264   INFO  ckpt             checkpoint_epoch_80.pth
2023-07-05 15:07:19,265   INFO  pretrained_model None
2023-07-05 15:07:19,265   INFO  launcher         none
2023-07-05 15:07:19,265   INFO  tcp_port         29051
2023-07-05 15:07:19,266   INFO  sync_bn          False
2023-07-05 15:07:19,266   INFO  fix_random_seed  True
2023-07-05 15:07:19,266   INFO  ckpt_save_interval 1
2023-07-05 15:07:19,267   INFO  local_rank       0
2023-07-05 15:07:19,267   INFO  max_ckpt_save_num 30
2023-07-05 15:07:19,268   INFO  merge_all_iters_to_one_ep

2023-07-05 15:07:19.210503
Using GPU:0


2023-07-05 15:07:19,423   INFO  Database filter by min points Car: 14357 => 13532
2023-07-05 15:07:19,424   INFO  Database filter by min points Pedestrian: 2207 => 2168
2023-07-05 15:07:19,425   INFO  Database filter by min points Cyclist: 734 => 705
2023-07-05 15:07:19,440   INFO  Database filter by difficulty Car: 13532 => 10759
2023-07-05 15:07:19,442   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2023-07-05 15:07:19,443   INFO  Database filter by difficulty Cyclist: 705 => 581
2023-07-05 15:07:19,447   INFO  Loading KITTI dataset
2023-07-05 15:07:19,530   INFO  Total samples for KITTI dataset: 3712
2023-07-05 15:07:19,531   INFO  **********************Start evaluation kitti_models/pointpillar(default)**********************
2023-07-05 15:07:19,533   INFO  Loading KITTI dataset
2023-07-05 15:07:19,616   INFO  Total samples for KITTI dataset: 3769
2023-07-05 15:07:19,618   INFO  ----------------Noise Experiment----------------


I-V_data_2um_length_200nm_diameter_NA_third_etch_18min_Pb_ED_3h_180C_MABr_no_200nm_Ag_memory_8V_probe.xlsx
0.42 0.23 0.11
#################### imported! ##########################


/home/pai/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-07-05 15:07:21,117   INFO  ==> Loading parameters from checkpoint checkpoint_epoch_80.pth to CPU
2023-07-05 15:07:21,196   INFO  ==> Loading optimizer parameters from checkpoint checkpoint_epoch_80.pth to CPU
2023-07-05 15:07:21,203   INFO  ==> Done
2023-07-05 15:07:21,205   INFO  **********************Start training kitti_models/pointpillar(default)**********************
2023-07-05 15:07:21,206   INFO  **********************End training**********************


==> Checkpoint trained from version: pcdet+0.3.0+26a1612
0.42 0.23 0.11
/mnt/workspace/sunqiao/OpenPCDet/tools/save_path/default/ckpt


2023-07-05 15:07:51,237   INFO  ---------------Epoch-1-Noise-I-V_data_2um_length_200nm_diameter_NA_third_etch_18min_Pb_ED_3h_180C_MABr_no_200nm_Ag_memory_8V_probe.xlsx-evaluate----------------
eval:   0%|          | 0/3769 [00:00<?, ?it/s]

KeyError: 'batch_size'

In [ ]:
file2ap_dict

In [ ]:
for f_name in sorted(hw_data_files):
    print(f_name)

In [ ]:
new_dict

In [ ]:
# import numpy as np

# # my_dict = file2sigma  # { 'Apple': 4, 'Banana': 2, 'Orange': 6, 'Grapes': 11}
# # # 保存文件
# # np.save('file2sigma.npy', my_dict)
# # 读取文件 
# # new_dict = np.load('./hardware_noise/file2sigma.npy', allow_pickle='TRUE').item()
# # print(new_dict)

# # for i, f_name in enumerate(new_dict.keys()):
# #     # if ~f_name.endswith('xlsx'):
# #     #     pass
# #     # if f_name != sorted(hw_data_files)[i+1]:
# #     #     print('!')
# #     for u in range(10):
# #         with open('sigma.txt', 'a+') as f:
# #             print(new_dict[f_name], file=f)
            
# new_dict = np.load('./hardware_noise/file2usability.npy', allow_pickle='TRUE').item()
# # print(new_dict)

# for i, f_name in enumerate(new_dict.keys()):
#     # if ~f_name.endswith('xlsx'):
#     #     pass
#     # if f_name != sorted(hw_data_files)[i+1]:
#     #     print('!')
#     for u in range(10):
#         with open('usability.txt', 'a+') as f:
#             print(new_dict[f_name], file=f)